# 暗号通貨価格予測モデル (Transformer)

このノートブックでは、改善されたTransformerモデルを使用して暗号通貨の価格予測を行います。
Google DriveとローカルCSVの両方からデータを読み込むことができます。

In [ ]:
# 必要なライブラリのインストール
!pip install tensorflow pandas numpy scikit-learn backtesting ta

In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# リポジトリのクローン
!git clone https://github.com/Kotakageyama/sakata_goho_chart.git
%cd sakata_goho_chart

In [ ]:
# 必要なモジュールのインポート
from src.data import CryptoDataLoader
from src.models.transformer_model import create_model
from src.models.training import ModelTrainer
from src.models.strategy import TransformerStrategy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## データの読み込み
Google DriveまたはローカルCSVからデータを読み込むことができます。

In [ ]:
# データローダーの初期化
data_loader = CryptoDataLoader()

# Google Driveからデータを読み込む場合
drive_path = "/content/drive/MyDrive/Colab Notebooks/DataSetBox/ETHUSD_2Year_2022-11-15_2024-11-15.csv"
df_drive = data_loader.load_data(drive_path)
print("Google Driveからのデータ形状:", df_drive.shape)
print("\nGoogle Driveデータのサンプル:")
display(df_drive.head())

# ローカルCSVからデータを読み込む場合
local_path = "data/ETHUSD_2Year_2022-11-15_2024-11-15.csv"
df_local = data_loader.load_data(local_path)
print("\nローカルCSVからのデータ形状:", df_local.shape)
print("\nローカルデータのサンプル:")
display(df_local.head())

## モデルのトレーニング

In [ ]:
# データの準備
sequence_length = 60
X_train, X_test, y_train, y_test, scaler = data_loader.prepare_data(
    df_drive,
    sequence_length=sequence_length
)

# モデルトレーナーの初期化
trainer = ModelTrainer(
    sequence_length=sequence_length,
    num_features=X_train.shape[2],
    n_splits=5
)

# ハイパーパラメータの最適化
param_grid = {
    "d_model": [64, 128],
    "num_heads": [4, 8],
    "num_transformer_blocks": [2, 4],
    "dropout": [0.1, 0.2]
}

best_params, best_metrics = trainer.optimize_hyperparameters(
    X_train,
    y_train,
    param_grid,
    epochs=50,
    batch_size=32,
    validation_split=0.2
)

print("最適なパラメータ:", best_params)
print("最良のメトリクス:", best_metrics)

## バックテストの実行

In [ ]:
from backtesting import Backtest

# モデルの予測を取得
predictions = trainer.best_model.predict(X_test)
price_pred = predictions[0]
direction_pred = predictions[1]

# バックテストデータの準備
test_data = df_drive.iloc[-len(y_test):].copy()
test_data["price_pred"] = price_pred
test_data["direction_pred"] = direction_pred

# バックテストの実行
bt = Backtest(
    test_data,
    TransformerStrategy,
    cash=10000,
    commission=.002,
    trade_on_close=True,
)

# 結果の表示
stats = bt.run()
print(stats)
bt.plot()